In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS steamdatabricks_workspace.bronze

In [0]:
%sql
CREATE EXTERNAL TABLE IF NOT EXISTS steamdatabricks_workspace.bronze.playtime (
  appid STRING,
  playtime_forever STRING,
  playtime_windows_forever STRING,
  playtime_mac_forever STRING,
  playtime_linux_forever STRING,
  playtime_deck_forever STRING,
  rtime_last_played STRING,
  playtime_disconnected STRING
)
USING PARQUET
LOCATION 'abfss://steam@steamstorageaccount.dfs.core.windows.net/bronze/playtime/'

In [0]:
%sql
CREATE EXTERNAL TABLE IF NOT EXISTS steamdatabricks_workspace.bronze.details (
  data STRUCT<
    about_the_game STRING,
    achievements STRUCT<
      highlighted ARRAY<STRUCT<
        name STRING,
        path STRING
      >>,
      total STRING
    >,
    background STRING,
    background_raw STRING,
    capsule_image STRING,
    capsule_imagev5 STRING,
    categories ARRAY<STRUCT<
      description STRING,
      id STRING
    >>,
    content_descriptors STRUCT<
      ids ARRAY<STRING>,
      notes STRING
    >,
    controller_support STRING,
    detailed_description STRING,
    developers ARRAY<STRING>,
    dlc ARRAY<STRING>,
    drm_notice STRING,
    ext_user_account_notice STRING,
    genres ARRAY<STRUCT<
      description STRING,
      id STRING
    >>,
    header_image STRING,
    is_free STRING,
    legal_notice STRING,
    linux_requirements STRING,
    mac_requirements STRING,
    metacritic STRUCT<
      score STRING,
      url STRING
    >,
    movies ARRAY<STRUCT<
      highlight STRING,
      id STRING,
      mp4 STRUCT<
        `480` STRING,
        max STRING
      >,
      name STRING,
      thumbnail STRING,
      webm STRUCT<
        `480` STRING,
        max STRING
      >
    >>,
    name STRING,
    package_groups ARRAY<STRUCT<
      description STRING,
      display_type STRING,
      is_recurring_subscription STRING,
      name STRING,
      save_text STRING,
      selection_text STRING,
      subs ARRAY<STRUCT<
        can_get_free_license STRING,
        is_free_license STRING,
        option_description STRING,
        option_text STRING,
        packageid STRING,
        percent_savings STRING,
        percent_savings_text STRING,
        price_in_cents_with_discount STRING
      >>,
      title STRING
    >>,
    packages ARRAY<STRING>,
    pc_requirements STRUCT<
      minimum STRING,
      recommended STRING
    >,
    platforms STRUCT<
      linux STRING,
      mac STRING,
      windows STRING
    >,
    price_overview STRUCT<
      currency STRING,
      discount_percent STRING,
      final STRING,
      final_formatted STRING,
      initial STRING,
      initial_formatted STRING
    >,
    publishers ARRAY<STRING>,
    ratings STRUCT<
      bbfc STRUCT<
        rating STRING,
        required_age STRING,
        use_age_gate STRING
      >,
      crl STRUCT<
        rating STRING
      >,
      csrr STRUCT<
        descriptors STRING,
        rating STRING
      >,
      dejus STRUCT<
        banned STRING,
        descriptors STRING,
        rating STRING,
        rating_generated STRING,
        required_age STRING,
        use_age_gate STRING
      >,
      esrb STRUCT<
        descriptors STRING,
        display_online_notice STRING,
        interactive_elements STRING,
        rating STRING,
        required_age STRING,
        use_age_gate STRING
      >,
      fpb STRUCT<
        descriptors STRING,
        rating STRING
      >,
      kgrb STRUCT<
        descriptors STRING,
        rating STRING
      >,
      mda STRUCT<
        rating STRING
      >,
      nzoflc STRUCT<
        descriptors STRING,
        rating STRING
      >,
      oflc STRUCT<
        descriptors STRING,
        rating STRING,
        required_age STRING,
        use_age_gate STRING
      >,
      pegi STRUCT<
        descriptors STRING,
        rating STRING,
        required_age STRING,
        use_age_gate STRING
      >,
      steam_germany STRUCT<
        banned STRING,
        descriptors STRING,
        rating STRING,
        rating_generated STRING,
        required_age STRING,
        use_age_gate STRING
      >,
      usk STRUCT<
        descriptors STRING,
        rating STRING,
        required_age STRING,
        use_age_gate STRING
      >
    >,
    recommendations STRUCT<
      total STRING
    >,
    release_date STRUCT<
      coming_soon STRING,
      date STRING
    >,
    required_age STRING,
    reviews STRING,
    screenshots ARRAY<STRUCT<
      id STRING,
      path_full STRING,
      path_thumbnail STRING
    >>,
    short_description STRING,
    steam_appid STRING,
    support_info STRUCT<
      email STRING,
      url STRING
    >,
    supported_languages STRING,
    type STRING,
    website STRING
  >,
  success STRING
)
USING PARQUET
LOCATION 'abfss://steam@steamstorageaccount.dfs.core.windows.net/bronze/details/'
